In [ ]:
"""
analyze_bias.py

Loads results/llm_outputs.csv and performs simple descriptive analysis
to detect patterns consistent with bias:
- How often each player is recommended under each condition
- Any imbalance across different framings
"""

from pathlib import Path
import pandas as pd

RESULTS_PATH = Path("results") / "llm_outputs.csv"
SUMMARY_DIR = Path("analysis")


def main():
    SUMMARY_DIR.mkdir(exist_ok=True)

    if not RESULTS_PATH.exists():
        raise FileNotFoundError(
            f"Could not find {RESULTS_PATH}. Make sure you have saved your model outputs."
        )

    df = pd.read_csv(RESULTS_PATH)

    # Basic sanity check
    print("Number of rows in results:", len(df))
    print("Unique hypotheses:", df["hypothesis"].unique())
    print("Unique conditions:", df["condition"].unique())
    print()

    # How often each player is recommended per hypothesis/condition
    if "recommended_player" in df.columns:
        group = (
            df.groupby(["hypothesis", "condition", "recommended_player"])
            .size()
            .reset_index(name="count")
            .sort_values(["hypothesis", "condition", "count"], ascending=[True, True, False])
        )
        print("Recommendation counts by hypothesis/condition:\n")
        print(group)

        out_path = SUMMARY_DIR / "recommendation_counts.csv"
        group.to_csv(out_path, index=False)
        print(f"\nSaved recommendation counts to {out_path.resolve()}")

    else:
        print("Column 'recommended_player' not found; skipping player-level analysis.")


if __name__ == "__main__":
    main()
